# Players Missed per Game due to Injury

In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *
from nba_api.stats.endpoints import commonallplayers, playerindex

player_dict = get_players_pbp()
teams_dict, teams_list = get_teams()

data_DIR = "../data/player_boxscores/"
data_DIR1 = "../data/injuries/"
box_DIR = "../data/box/"
fig_DIR = "../figs/team_leaders/"

In [ ]:
def is_injured(dfinj, pId_missed, game_date):
    missed_games = np.array([False] * len(pId_missed))
    for i,pId in enumerate(pId_missed):
        df_p = dfinj.query(f'playerID == {pId}').reset_index(drop=True)
        if len(df_p) > 0:
            df_p["Comp"] = df_p["Date"] <= game_date
            idxi = df_p[df_p["Comp"]].index
            if len(idxi) > 0:
                idx = idxi[-1]
                missed_game = df_p["Out"].loc[idx]
                missed_games[i] = missed_game
    gp = missed_games*pId_missed
    pId_m = gp[gp !=0 ]
    # pId_m = list(pId_m)
    return pId_m

In [ ]:
year = 2023

In [ ]:
dfinj = pd.read_parquet(data_DIR1 + f'NBA_prosptran_injuries_{year}.parquet')
start_date = dt.datetime(year=2023, month=7,day=1)
end_date = dt.datetime(year=2024, month=6,day=30)
today = datetime.today()
if (end_date - today).total_seconds() > 0:
    end_date = today
end_date = datetime(end_date.year, end_date.month, end_date.day)

In [ ]:
dfinjg = dfinj.groupby("playerID")
keys = list(dfinjg.groups)
dfia = []
for key in keys:
    dfi = dfinjg.get_group(key)
    dfi = pd.concat([dfi.head(1),dfi])
    if dfi["Date"].iloc[0] != start_date:
        dfi["Date"].iloc[0] = start_date
        dfi["In"].iloc[0] = False
        dfi["Out"].iloc[0] = False
        dfi["Notes"].iloc[0] = "Start of Season"
    dfi["Date1"] = dfi["Date"].shift(periods=-1)
    dfi["Date1"].iloc[-1] = end_date
    dfi['Dater'] = dfi.apply(lambda row: pd.date_range(row["Date"], row['Date1'], freq='D'), axis=1)
    dfj = dfi.explode('Dater').reset_index(drop=True) \
            .drop(columns=['Date', 'Date1']) \
            .rename(columns={'Dater': 'Date'}) 
    dfj.insert(0,"Date",dfj.pop("Date"))
    dfia.append(dfj)
dfk = pd.concat(dfia)

In [ ]:
dfk

In [ ]:
# %%timeit
def analyze_injuries(year):
# load team boxscores
    season_str = str(year) + "-" + str(year+1)[-2:]
    df0 = pd.read_parquet(box_DIR + f"NBA_BOX_T_Base_{year}.parquet")
    df0= df0[["GAME_ID","TEAM_ID","GAME_DATE","MATCHUP","WL","PLUS_MINUS"]]
    df0["GAME_ID"] = df0["GAME_ID"].astype(int)
    # load player indvidual game boxscores
    df1 = pd.read_parquet(box_DIR + "NBA_BOX_P_" + "Base" + "_" + str(year) + ".parquet")
    df1["GAME_DATE"] = pd.to_datetime(df1["GAME_DATE"], format="%Y-%m-%d")
    games_list = list(df1["GAME_ID"].unique())
    # load injury database
    dfinj = pd.read_parquet(data_DIR1 + f'NBA_prosptran_injuries_{year}.parquet')
    # load team rosters
    # dfr = commonallplayers.CommonAllPlayers(league_id = "00", season =year, is_only_current_season=True).get_data_frames()[0]
    # dfr = dfr.query("ROSTERSTATUS == 1 & GAMES_PLAYED_FLAG == 'Y'")
    dfr = playerindex.PlayerIndex(league_id = "00", season =season_str).get_data_frames()[0]
    dfr = dfr[(dfr["ROSTER_STATUS"] == 1)].reset_index(drop=True)
    dfr = dfr.rename(columns={"PERSON_ID":"PLAYER_ID"})
    dfr = dfr[["TEAM_ID","PLAYER_ID"]]
    dfrt = dfr.groupby("TEAM_ID")
    # group player boxscores by gameID and TeamID
    df2 = df1.groupby(["GAME_ID","TEAM_ID"])
    # loop through all groups and get injured players 
    keys = list(df2.groups)
    dfma = []
    # i=0
    for key in tqdm(keys):
        p_played =  df2["PLAYER_ID"].get_group(key).to_numpy()
        p_roster = dfrt["PLAYER_ID"].get_group(key[1]).to_numpy()
        # pId_missed = np.array(list(set(p_roster) - set(p_played)))#.astype(int)
        pId_missed = np.setdiff1d(p_roster,p_played)
        game_date = df2["GAME_DATE"].get_group(key).iloc[0]
        players_missed = is_injured(dfinj,pId_missed,game_date)
        dfm1 = pd.DataFrame({"PLAYER_ID":players_missed})
        dfm1["TEAM_ID"] = key[1]
        dfm1["GAME_ID"] = key[0]
        dfm1["GAME_DATE"] = game_date
        dfma.append(dfm1)
        # i+=1
        # if i > 50:
        #     break
    dfm = pd.concat(dfma)
    dfm["PLAYER_ID"] = dfm["PLAYER_ID"].astype(int)
    # map player and team names
    dfm["PLAYER_NAME"] = dfm["PLAYER_ID"].map(player_dict)
    dfm["TEAM_NAME"] = dfm["TEAM_ID"].map(teams_dict)
    # count number of players missed per game
    dfmg = dfm.groupby(["GAME_ID","TEAM_ID"])["PLAYER_ID"].count()
    dfmg = pd.DataFrame(dfmg)
    dfmg = dfmg.reset_index().rename(columns={"PLAYER_ID":"Players_Out"})
    dfmg["GAME_ID"] = dfmg["GAME_ID"].astype(int)
    # merge team boxscores to get game details like date and matchup
    dfmf = pd.merge(df0,dfmg,on=["GAME_ID","TEAM_ID"])
    dfmf["Team"] = dfmf["TEAM_ID"].map(teams_dict)
    dfmf = dfmf.drop(columns=["GAME_ID","TEAM_ID"])
    dfmf.insert(1,"Team",dfmf.pop("Team"))
    df_exp = dfmf.nlargest(10,"Players_Out")
    df_exp = df_exp.reset_index(drop=True).reset_index()
    df_exp["index"] += 1
    # get games with more than 5 players missing and win loss
    dfmf1 = dfmf.copy()
    dfmf1["Win"] = (dfmf["WL"] == "W")
    dfmf1["Loss"] = (dfmf["WL"] == "L")
    df_po = dfmf1.query("Players_Out >=5").groupby("Team").agg({"Win":["sum"],"Loss":["sum"],"Players_Out":["count"]})
    df_po.columns = ["Wins","Losses","Games"]
    df_po = df_po.sort_values(by="Games",ascending=False).reset_index().reset_index()
    df_po["index"] += 1
    df_x = dfmf1.groupby("Team").agg({"Players_Out":["sum"]})
    df_x.columns = ["Players_Out"]
    df_x = (df_x
            .reset_index()
            .sort_values("Players_Out",ascending=False)
            .reset_index(drop=True)
            .reset_index()
        )
    df_x["index"] += 1
    # df_x["Season"] = year
    return df_exp, df_po, df_x,dfmf1, dfm

# Single Season

In [ ]:
year = 2023
df_exp, df_po, df_x, dfmf, dfm = analyze_injuries(year)

In [ ]:
df_teams = pd.read_csv("../data/NBA_teams_colors_logos.csv")
df_y = pd.merge(df_x, df_teams, left_on="Team", right_on="nameTeam")
teams = df_y["Team"].to_list()
teams.reverse()
df_y["Team"] = pd.Categorical(df_y['Team'], categories=teams)

In [ ]:
fig_DIR

In [ ]:
p = (
    ggplot(df_y, aes(x='Team', y='Players_Out'))
    + geom_bar(aes(fill="colorsTeam"),stat="identity", alpha=0.8)
    + coord_flip()
    + scale_color_identity(aesthetics=["fill"])
    + theme_idv
    + theme(
        figure_size=(10,8),
        plot_title=element_text(size=20, weight="bold")
    )
    + labs(
        title="Player Games Missed",
        subtitle="Players missing the game due to injury/personal reasons | Injury data updated till Jan 25, 2024",
        caption="@SravanNBA | Source: nba.com/stats, prosportstransactions",
        y= "Player Games Missed",
        x="",
    )
)
p.save(fig_DIR + f"Player Games Missed.png", dpi=300)
p

In [ ]:
dfm.query('TEAM_NAME == "Memphis Grizzlies"')["PLAYER_NAME"].unique()

In [ ]:
# dfm.query("TEAM_NAME == 'Memphis Grizzlies'").head()
# dfmf.query("Team == 'Los Angeles Lakers'")
# df_exp.head()
# df_po.sum()

In [ ]:
%reload_ext rpy2.ipython

In [ ]:
%%R -i df_exp
library(tidyverse)
library(gt)
library(gtExtras)
df <- df_exp
df %>% 
  gt()%>%
  tab_header(
    title = md("**Most Players Out during a Game in 2023-24**"),
    subtitle = "Players Out: Players missing the game due to injury/personal reasons"  
    ) %>%
    data_color(columns = c(PLUS_MINUS), palette = c("red","green")) %>%
    cols_align(align = "center")  %>%
    cols_align(align = "left",columns = c(GAME_DATE,Team,MATCHUP))  %>%
    cols_label(
      index = "#",GAME_DATE = "Date", MATCHUP = "Matchup", PLUS_MINUS = "+/-", Players_Out ="Players Out"
    ) %>%
    tab_options(
        table.background.color = "floralwhite",
        column_labels.font.size = 12,
        column_labels.font.weight = 'bold',
        row_group.font.weight = 'bold',
        row_group.background.color = "#E5E1D8",
        table.font.size = 10,
        heading.title.font.size = 16,
        heading.subtitle.font.size = 10,
        table.font.names = "Consolas", 
        data_row.padding = px(6)
    ) %>% 
    # tab_source_note(
    # source_note = "Min Miss PG (Per Game) = Min Missed/GP (Games Played) | Min Miss % = Min Miss PG/240 Min (240=48x5)  ")  %>% 
    tab_source_note(
    source_note = "@SravanNBA | Source: nba.com/stats, prosportstransactions" ) %>% gtsave("../figs/team_leaders/players_missed.png",size=5) 

In [ ]:
%%R -i df_x
df <- df_x
library(tidyverse)
library(gt)
library(gtExtras)
df %>% 
  gt()%>%
  tab_header(
    title = md("**Most Player Games Missed 2023-24 NBA Season**"),
    subtitle = "Players Out: Players missing the game due to injury/personal reasons"  
    ) %>%
    cols_align(align = "center")  %>%
    cols_align(align = "left",columns = c(Team))  %>%
    gt_plt_bar(column = Players_Out, keep_column = TRUE, color = "red") %>%
    cols_label(
      index = "#", Players_Out = "Games Missed"
    ) %>%
    tab_options(
        table.background.color = "floralwhite",
        column_labels.font.size = 12,
        column_labels.font.weight = 'bold',
        row_group.font.weight = 'bold',
        row_group.background.color = "#E5E1D8",
        table.font.size = 10,
        heading.title.font.size = 16,
        heading.subtitle.font.size = 10,
        table.font.names = "Consolas", 
        data_row.padding = px(6)
    ) %>% 
    # tab_source_note(
    # source_note = "Min Miss PG (Per Game) = Min Missed/GP (Games Played) | Min Miss % = Min Miss PG/240 Min (240=48x5)  ")  %>% 
    tab_source_note(
    source_note = "@SravanNBA | Source: nba.com/stats, prosportstransactions" ) %>% gtsave("../figs/team_leaders/player_games_missed.png",size=5) 

In [ ]:
%%R -i df_po
df <- df_po
library(tidyverse)
library(gt)
library(gtExtras)
df %>% 
  gt()%>%
  tab_header(
    title = md("**Games with 5 or more Players Out in 2023-24**"),
    subtitle = "Players Out: Players missing the game due to injury/personal reasons"  
    ) %>%
    cols_align(align = "center")  %>%
    cols_align(align = "left",columns = c(Team))  %>%
    cols_label(
      index = "#", pl
    ) %>%
    gt_plt_bar(column = Games, keep_column = TRUE, color = "red") %>%
    tab_options(
        table.background.color = "floralwhite",
        column_labels.font.size = 12,
        column_labels.font.weight = 'bold',
        row_group.font.weight = 'bold',
        row_group.background.color = "#E5E1D8",
        table.font.size = 10,
        heading.title.font.size = 16,
        heading.subtitle.font.size = 10,
        table.font.names = "Consolas", 
        data_row.padding = px(6)
    ) %>% 
    # tab_source_note(
    # source_note = "Min Miss PG (Per Game) = Min Missed/GP (Games Played) | Min Miss % = Min Miss PG/240 Min (240=48x5)  ")  %>% 
    tab_source_note(
    source_note = "@SravanNBA | Source: nba.com/stats, prosportstransactions" ) %>% gtsave("../figs/team_leaders/games_missed_5more.png",size=5) 

# Multi Seasons

In [ ]:
dfa = []
for year in range(2018,2023):
    print(year)
    df_exp, df_po, df_x = analyze_injuries(year)
    dfa.append(df_x)
dfs = pd.concat(dfa)

In [ ]:
dfs